In [22]:
###Import standard packages###
import numpy as np
from scipy import io
from matplotlib import pyplot as plt
import pickle

###Import functions for binning data for preprocessing###
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output
from Neural_Decoding.decoders import LSTMDecoder
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

In [23]:
bins_before=13 #How many bins of neural data prior to the output are used for decoding
bins_current=1 #Whether to use concurrent time bin of neural data
bins_after=0 #How many bins of neural data after the output are used for decoding

In [24]:
###Load Data###
folder='/Users/kendranoneman/Projects/mayo/Neural_Decoding/example_data/' #ENTER THE FOLDER THAT YOUR DATA IS IN
# folder='/Users/jig289/Dropbox/MATLAB/Projects/In_Progress/BMI/Processed_Data/' 
data=io.loadmat(folder+'s1_data_raw.mat')
spike_times=data['spike_times'] #Load spike times of all neurons
vels=data['vels'] #Load x and y velocities
vel_times=data['vel_times'] #Load times at which velocities were recorded

In [25]:
dt=.05 #Size of time bins (in seconds)
t_start=vel_times[0] #Time to start extracting data - here the first time velocity was recorded
t_end=vel_times[-1] #Time to finish extracting data - here the last time velocity was recorded
downsample_factor=1 #Downsampling of output (to make binning go faster). 1 means no downsampling.

In [26]:
#When loading the Matlab cell "spike_times", Python puts it in a format with an extra unnecessary dimension
#First, we will put spike_times in a cleaner format: an array of arrays
spike_times=np.squeeze(spike_times)
for i in range(spike_times.shape[0]):
    spike_times[i]=np.squeeze(spike_times[i])

In [27]:
###Preprocessing to put spikes and output in bins###

#Bin neural data using "bin_spikes" function
neural_data=bin_spikes(spike_times,dt,t_start,t_end)

#Bin output (velocity) data using "bin_output" function
vels_binned=bin_output(vels,vel_times,dt,t_start,t_end,downsample_factor)

In [28]:
data_folder='/Users/kendranoneman/Projects/mayo/Neural_Decoding/example_data/' #FOLDER YOU WANT TO SAVE THE DATA TO

with open(data_folder+'example_data_s1.pickle','wb') as f:
    pickle.dump([neural_data,vels_binned],f)

In [29]:
# Generate some fake trial data
n_trials = 250
n_neurons = 10
n_bins_per_trial = 50

# And generate some fake neural recordings
# We'll pretend we have a drift diffusion model

def generate_neural_data(n_trials, n_neurons = 25, n_bins_per_trial = 50,
                         noise_variance = 1, drift_rate = 0.07,
                         mean_rate = 25):
  """Generates fake neural data of shape (n_trials, n_neurons, n_bins_per_trial
        according to a drift diffusion process with given parameters.
     Also generates decisions, which is 0 or 1 depending on the "animal's decision"
        and is returned as an array of shape (n_trials,) 
        
    Returns: (neural_data, decisions)
  """

  decisions = np.random.binomial(1,.5,size = n_trials)
  
  neural_recordings = np.zeros((n_trials,n_neurons,n_bins_per_trial))

  for t in range(n_bins_per_trial):
    if t==0:
      neural_recordings[:,:,t] = mean_rate + np.random.randn(n_trials,n_neurons) * noise_variance
    else:
      neural_recordings[:,:,t] = neural_recordings[:,:,t-1] \
                      + np.reshape(drift_rate*(decisions*2-1),(len(decisions),1)) \
                      + np.random.randn(n_trials,n_neurons) * noise_variance
      
  return neural_recordings, decisions

neural_recordings, decisions = generate_neural_data(n_trials, 
                                         n_neurons, n_bins_per_trial)

In [30]:
# Let's do new electrophysiology and take new data. This cost $2,000,000 in 
# NIH funding so it better work.
new_neural_recordings, new_decisions = generate_neural_data(n_trials, 
                                             n_neurons, n_bins_per_trial)
new_X = np.reshape(new_neural_recordings, (n_trials,-1))


# Now you: score the model on this new data

In [31]:
# Function to get the covariate matrix that includes spike history from previous bins
X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

In [32]:
#Declare model and set parameters of the model
model_lstm=LSTMDecoder(units=400,num_epochs=5)

#Fit model
model_lstm.fit(X_train,y_train)

#Get predictions
y_test_predicted_lstm=model_lstm.predict(X_test)

NameError: name 'X_train' is not defined